# Documentation  

Anaconda 2019.07 버전의 python 3.7.4 defalut 값으로 진행  
  
Python = 3.7.4  
Pandas = 0.25.1   
Numpy = 1.16.5  
urllib3 = 1.24.2  
BeautifulSoup = 4.8.0  
requests = 2.22.0   
selenium = 3.141.0  

In [1]:
#import module

#python defalut module
import time
import re
import os
from datetime import datetime,timedelta
from IPython.display import Image

import pandas as pd
import numpy as np

import urllib3
from bs4 import BeautifulSoup
import requests

import selenium

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

import warnings
warnings.filterwarnings(action='ignore')

### 다른 사이트의 경우 IT 이외의 과학 뉴스 등 다른 분야의 기사가 많이 혼재하여
> 네이버 뉴스 IT/과학 헤드라인  
>    
> ITWorld korea의 머신러닝, 클라우드, 빅데이터 파트  
>    
> ZDnet의 인공지능 파트에서 데이터를 추출하였습니다.

<p> <img src="img/그림2.png" height="300px" width="700px"></p>

#### - 네이버 뉴스의 경우 "헤드라인 더보기" 전까지 기사의 제목과 URL을 추출후 DataFrame안에 저장하여 기록하고  
  
#### - 이후 저장한 URL에 개별 접속하여 기사의 본문을 가져와서 저장하는 형식으로 진행하였습니다

In [2]:
# 이전 문서와 중복되면 해당 기사의 row index를 기록하는 코드
def dupliacate_title(prev_list,prev_title) :
    if now_news in prev_title :
        prev_list.append(row) 


In [3]:
news = 'https://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=105' ## IT/과학 헤드라인 뉴스  url

res = requests.get(news, headers={'User-Agent':'Chrome/85.0.4183.121'},verify = False) 
# requests이용하여 뉴스 주소 Load, 네이버 크롬 차단 방지 위해 chrome version을 입력하고 뉴스 주소를 설정


# SSL ERROR 방지를 위해 Verify False 설정
soup = BeautifulSoup(res.text, "html.parser") # html기준으로 data parsing

title_list =[] # 제목을 저장하기 위한 list 생성
url_list = [] # URL을 저장하기 위한 list 생성
date_list = [] # 날짜를 저장하기 위한 list 생성

results = soup.select("#main_content a")  # 기사 제목과 URL 모두 html main_content a에 저장되어 있음. 해당 부분을 select하여 추출


# 제목, 링크, date 출력 ---------------------------
for result in results:
    if result.string == None: # 값이 존재하지 않을 경우 해당 루프 건너 뜀
        continue
    
    if result.attrs['href'] in ['#'] : # 헤드라인 더보기의 end code가 #이므로 #일 경우 loop문을 Break하여 종료함
        break
    
    if result.attrs['href'].startswith('http') == False : 
        # 해당 href에서 url 코드가 http로 시작하지 않을경우 url이 아니라고 판단하여 해당 부분 skip
        continue
    

    title_list.append(str.strip(result.string)) # 크롤링한 기사 제목을 리스트에 추가
    url_list.append(result.attrs['href']) # 크롤링한 기사 URL을 리스트에 추가
    date_list.append(datetime.today().strftime("%Y-%m-%d")) # 현재 날짜를 리스트에 추가


df = pd.DataFrame({'title': title_list,'url':url_list, 'date':date_list}) # 기사제목, URL, 날짜를 dataframe에 저장
df['title'] = df.title.str.replace('\\',"") # 작은 따옴표가  \' 형태로 기록되어 \를 공란으로 대체
df.head()

,title,url,date
0,"""아이폰12 배터리, 5G 환경선 안드로이드폰 못 미쳐”",https://news.naver.com/main/read.nhn?mode=LSD&...,2020-10-22
1,통신3사 ‘아이폰12’ 사전예약 혜택 어디가 좋나,https://news.naver.com/main/read.nhn?mode=LSD&...,2020-10-22
2,"KT, '아이폰12' 23일 사전예약 시작…예비부부 결합할인",https://news.naver.com/main/read.nhn?mode=LSD&...,2020-10-22
3,아이폰12 어디서 사야 빨리 쥘까… 사전예약 총정리,https://news.naver.com/main/read.nhn?mode=LSD&...,2020-10-22
4,페이팔 가상화폐 거래 소식에 관련주 우리기술투자 급등,https://news.naver.com/main/read.nhn?mode=LSD&...,2020-10-22


<p> <img src="img/그림1.png" height="300px" width="700px"></p>

### 위의 그림처럼 네이버는 기사들이 중복되어 나타나기 때문에 이를  제거하는 작업이 필요합니다
### 중복을 없애기 위해 기사 제목을 통해 중복 뉴스를 찾아 제거하였습니다

In [4]:
# 중복된 기사 list 추출
raw_df = df.copy()
duplicate_list = []
dup_title = soup.find_all("li",{"class":"cluster_item as_line"})
for dup in dup_title:
    text = dup.a.get_text()
    duplicate_list.append(text.replace('\\',""))
duplicate_list

['통신3사 ‘아이폰12’ 사전예약 혜택 어디가 좋나',
 "KT, '아이폰12' 23일 사전예약 시작…예비부부 결합할인",
 '아이폰12 어디서 사야 빨리 쥘까… 사전예약 총정리',
 '[강석기의 과학카페] 바이러스의 창 막는 박테리아의 방패',
 '[강석기의 과학카페] 장수말벌, 북아메리카 접수할까',
 '[강석기의 과학카페]100억년 뒤 태양계는 어떤 모습일까',
 '[아하! 우주] 소행성 베누의 흙 가져올까?…美 탐사선, 로봇팔 ‘쭈욱’ 카운트다운',
 '[아하! 우주] 아기가 아기를 낳다…아기별과 함께 크는 아기 행성 포착',
 '[아하! 우주] 화성 속살을 들여다볼 로버 퍼서비어런스의 비밀 무기']

#### 위에 기사 제목과 비교하면 중복된 기사인 1,2,3 번의 뉴스가 제거된 것을 볼 수 있습니다

또한 main topic인 기사의 경우 중복제거를 위해    
동일한 위치에 기록되는 것을 바탕으로 중복 제거를 시행하였습니다.

In [5]:
# 중복된 기사 제거
title_row = []
for row, title in enumerate(df['title']) :
    if title in duplicate_list :
        title_row.append(row)

if len(df) != 0 :
    
    df.drop(title_row,inplace=True)
    df.drop([5,7,9,11],inplace=True)
    df.reset_index(drop=True,inplace=True) # dataframe index 재설정
    df.head()

### 또한 헤드라인 뉴스안에는 전날에 생성된 헤드라인 뉴스도 포함되어 있기 때문에
### 전날의 뉴스를 저장하고 오늘 기사와 비교하여 중복된 기사를 제거하였습니다

In [6]:
# 이전 뉴스 기사 제목 Load
save_news = pd.read_csv('C:/driver/news_save/save_news.csv',encoding='CP949')
save_news.head()

,title,url,date
0,미 탐사선 소행성 베누에 10초간 터치다운… 태양계 비밀 풀리나,https://news.naver.com/main/read.nhn?mode=LSD&...,2020-10-21
1,"NASA 탐사선, 소행성 '베누'에 착륙해 토양 샘플 채취",https://news.naver.com/main/read.nhn?mode=LSD&...,2020-10-21
2,"미 탐사선, 소행성 '베누'서 토양 채취…태양계 형성 역사 규명 기대",https://news.naver.com/main/read.nhn?mode=LSD&...,2020-10-21
3,"NASA 우주 탐사선 오시리스-렉스, 소행성 베누 10초 '키스'",https://news.naver.com/main/read.nhn?mode=LSD&...,2020-10-21
4,5G 평균 속도 세계 2위인데…5G 가입자의 7%는 ‘LTE 리턴’,https://news.naver.com/main/read.nhn?mode=LSD&...,2020-10-21


In [7]:
today = datetime.today().strftime("%Y-%m-%d")
today = pd.to_datetime(today, format='%Y-%m-%d')
drop_list = []
prev_list = []
for row, prev_date in enumerate(save_news.date) :
    prev_date = pd.to_datetime(prev_date, format='%Y-%m-%d')
    days= str(today-prev_date)
    days = int(days[:1])
    if days >= 5 :
        drop_list.append(row)
        continue

save_news = save_news.drop(drop_list).reset_index(drop=True)
        
prev_title = save_news.title.tolist()        
        
for row, now_news in enumerate(df.title) :
    dupliacate_title(prev_list,prev_title)
        
df.drop(prev_list,inplace=True)

In [8]:
# # 데이터 저장
save_news = pd.concat([save_news,df,raw_df]) # 이전 뉴스에 오늘자 뉴스 추가
save_news.drop_duplicates(subset=['title'], keep='first',inplace=True)
save_news.to_csv('C:/driver/news_save/save_news.csv',encoding='CP949',index=False) # 추가한 데이터 저장

#### - 네이버 뉴스는 IT/과학이기 때문에 과학과 관련된 단어 리스트 추가를 통해
#### - 과학과 관련된 뉴스를 제거하였습니다

In [9]:
df

,title,url,date
0,"""아이폰12 배터리, 5G 환경선 안드로이드폰 못 미쳐”",https://news.naver.com/main/read.nhn?mode=LSD&...,2020-10-22
2,아이언맨 비서처럼… 옷 추천해주는 AI 패션 코디,https://news.naver.com/main/read.nhn?mode=LSD&...,2020-10-22


In [10]:
science_list = ['노벨', '화학','과학자','후쿠시마','세포','방사능','방사선','세포','암','지렁이','독성','플라스틱'
                '[이슈IN]','[슬기로운 소비생활]','[사이언스샷]','[핵잼 사이언스]','[사이언스카페]','[이덕환의 과학세상]',
                '[사이언스]','[강석기의과학카페]','바이러스','미생물','가위', '세포의','코로나','코로나19','초미세',
                '유체','개체가','초미세먼지','유전자','유전자가위로','치료제', '소행성' ,'태양계' ,'탐사선', '토양',
                'DNA','오염수','국감','세슘','원전','미세먼지'] # 1과학 단어 list

In [11]:
df

,title,url,date
0,"""아이폰12 배터리, 5G 환경선 안드로이드폰 못 미쳐”",https://news.naver.com/main/read.nhn?mode=LSD&...,2020-10-22
2,아이언맨 비서처럼… 옷 추천해주는 AI 패션 코디,https://news.naver.com/main/read.nhn?mode=LSD&...,2020-10-22


In [12]:
science_drop = []
for row, science in enumerate(df.title) :
    count=0
    title_split = re.sub("[^ㄱ-ㅎ ㅏ-ㅣ 가-힣| a-zA-Z0-9]", " ",science) # 특수문자 제거
    title_split = re.split(' ', title_split) # 기사 제목을 띄어쓰기 단위로 구분
    title_split = [ string for string in title_split if string != ''] #  리스트에 빈공간인 ''로 저장되는 list 삭제
    
    for k in science_list :
        if title_split.count(k)>=1 :
            count +=1
            if count>=2 :
                count=0 
                science_drop.append(row)
                break
science_drop
df.drop(science_drop,inplace=True)

In [13]:
df

,title,url,date
0,"""아이폰12 배터리, 5G 환경선 안드로이드폰 못 미쳐”",https://news.naver.com/main/read.nhn?mode=LSD&...,2020-10-22
2,아이언맨 비서처럼… 옷 추천해주는 AI 패션 코디,https://news.naver.com/main/read.nhn?mode=LSD&...,2020-10-22


### 크롤링한 기사 URL에 접속하여 제목과 본문을 추출하여 저장하였습니다.

In [14]:
news_url =df['url'].tolist()  ## 각 뉴스 URL 주소 입력
article_list = [] # 뉴스 본문 저장 LIST 생성

for page in news_url :
    
    res = requests.get(page, headers={'User-Agent':'Chrome/85.0.4183.121'},verify = False) ## 뉴스 URL 주소 접속
    news_html = BeautifulSoup(res.text,'html.parser') # html 태그 기준으로 parsing
    content = news_html.select("#articleBodyContents") # 기사 본문 저장되어 있는 html주소 복사


    news_content = content[0] # select된 html중 기사는 0번째 리스트에 존재하며 이를 저장


    for target in news_content.find_all('script'): # 기사 본문에 추가로 저장되어 있는 각 class들 제거
        target.extract()

    for target in news_content.find_all('a'): # 기사 본문에 추가로 저장되어 있는 각 class들 제거
        target.extract()

    for target in news_content.find_all('div'): # 기사 본문에 추가로 저장되어 있는 각 class들 제거
        target.extract()

    for target in news_content.find_all('iframe'): # 기사 본문에 추가로 저장되어 있는 각 class들 제거
        target.extract()


    for target in news_content.find_all('strong'): # 기사 본문에 추가로 저장되어 있는 각 class들 제거
        target.extract()


    final_article = str(news_content).replace('&amp;apos;','').replace('\n','').replace('\t','').replace('▶','').replace('img_desc">','img_desc"> <br/><br/>')
    # 전처리한 네이버 기사 저장
    article_list.append(final_article)
    # 3 곳의 기사를 통합하기 위해 통합 List에 저장
    time.sleep(10)

final_title = df.title.tolist()

## ITWorld Korea 머신러닝, 클라우드, 빅데이터 태그 크롤링

> ITWorld 날짜의 경우 기사 게시 일주일 이전에는 n분 전, n 시간 전, n일 전, 일주일 후에는 yyyy.mm.dd로 변경되기 때문에  
> 4가지 경우에 따라 데이터를 추출하는 작업이 필요합니다   
>  
> 해당 자료를 모두 정제하여 파이썬 표준 datetime인 yyyy-mm-dd로 변경하였습니다
> 

In [15]:
itw_link = ['http://www.itworld.co.kr/t/54649/%EB%B9%85%20%EB%8D%B0%EC%9D%B4%ED%84%B0',
               'http://www.itworld.co.kr/t/69500/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D',
               'http://www.itworld.co.kr/t/34/%ED%81%B4%EB%9D%BC%EC%9A%B0%EB%93%9C']
               ## ITWorld 머신러닝, 클라우드, 빅데이터 tag URL주소
it_title_list =[] # 제목을 저장하기 위한 list 
it_url_list = [] # URL을 저장하기 위한 list 
it_total_date = [] # 닐짜를 저장하기 위한 list 
tag_list = [] # tag를 저장하기 위한 list    
    
for itw in itw_link :
    res = requests.get(itw, headers={'User-Agent':'Chrome/85.0.4183.121'},verify = False) 
    # requests이용하여 뉴스 주소 Load, 네이버 크롬 차단 방지 위해 chrome version을 입력하고 뉴스 주소를 설정
    # SSL ERROR 방지를 위해 Verify False 설정
    soup = BeautifulSoup(res.text, "html.parser") # html기준으로 parsing


    # 제목, 링크 출력 ---------------------------(2)
    it_title = soup.select('div > h4 > a') # 제목 링크가 존재하는 tag
    for i in it_title :
        it_title_list.append(str.strip(i.string)) #기록된 기사 제목을 리스트에 추가
        itworld_url = 'http://www.itworld.co.kr'+ i.attrs['href'] # 기사 url 추출
        it_url_list.append(itworld_url) # 기사 url 저장


    # # 날짜 추출

    it_date = soup.find_all("div",{"class":"news_list_time default_small_font color_lightest_gray"})
    # 날짜가 기록된 태그 저장
    start_date = datetime.now() # 현재 날짜 저장

    for j in it_date :
        it_date_list = j.get_text() #날짜 추출

        if '시간' in it_date_list : # 날짜가 n시간 전으로 표기될 경우 문자열 처리
            date_find = re.findall(r'\d+', it_date_list) 
            final_date = str(datetime.now() - timedelta(hours=int(date_find[0])))
            it_total_date.append(final_date)

        elif '분' in it_date_list : # 날짜가 n분 전으로 표기될 경우 문자열 처리
            date_find = re.findall(r'\d+', it_date_list)
            final_date = str(datetime.now() - timedelta(minutes=int(date_find[0])))
            it_total_date.append(final_date)

        elif '일' in it_date_list: # 날짜가 n일 전으로 표기될 경우 문자열 처리
            date_find = re.findall(r'\d+', it_date_list)
            final_date = str(datetime.now() - timedelta(days=int(date_find[0])))
            it_total_date.append(final_date)    

        elif bool(re.search(r'\d{3,4}.\d{1,2}.\d{2}',it_date_list)) == True : # yyyy.mm.dd로 표기될 경우 처리
            final_date = it_date_list.replace('.',"-")
            it_total_date.append(final_date)
            
    ## tag 추출
    itw_tag = soup.find_all("div",{"class":"color_lightest_gray"}) # 각 기사 타입을 추출하는 코드

    for tag in itw_tag :
        tag_seg = re.findall(r'\d+', tag.get_text())
        if not tag_seg:
            tag = tag.get_text()
            tag = tag.replace('\n','').replace('\t','').replace(' ','')
            tag_list.append(tag)



### ITWorld의 머신러닝, 클라우드, 빅데이터 중 두 곳 이상에 모두 게시된  중복 기사가 존재합니다  
### 이를 위해 세 군데에서 수집한 기사를 모두 한 곳으로 모아 중복을 제거하였습니다
### 또한 네이버 기사와 마찬가지로 전날과 중복 방지를 위해 이전에 저장한 기사를 통해 중복을 제거하였습니다

In [16]:
itw_df = pd.DataFrame({'title': it_title_list,'url':it_url_list,'date':it_total_date,'tag':tag_list}) 
# 기사제목과, URL을 dataframe에 임시 저장
itw_df['title'] = itw_df.title.str.replace('\\',"") # 따옴표가  \' 형태로 기록되어 해당 \를 공란으로 대체
itw_df['date'] = itw_df['date'].str[:11].str.strip() # 날짜의 앞뒤 공백 제거
itw_df = itw_df[~itw_df['title'].str.contains('MOU',na=False)] # 유용하지 않은 기사인 MOU 제거
itw_df.drop_duplicates(subset=['title'], keep='last',inplace=True) # 기사 중 마지막 기사만 남기고 중복 제거
itw_df = itw_df.dropna().reset_index(drop=True) # 데이터 편집을 위한 NA제거 및 index reset
itw_df.head()

,title,url,date,tag
0,"뉴패러다임인베스트먼트, 빅데이터 기반 중고차 플랫폼 ‘카툴’ 시드 투자",http://www.itworld.co.kr/t/54649/%EB%B9%85%20%...,2020-10-21,빅데이터
1,"효성인포메이션시스템, 오브젝트 스토리지 활용한 하둡 데이터 최적화 솔루션 제공",http://www.itworld.co.kr/t/54649/%EB%B9%85%20%...,2020-10-21,빅데이터/스토리지
2,"트레저데이터, 한국 법인 공식 출범",http://www.itworld.co.kr/t/54649/%EB%B9%85%20%...,2020-10-08,빅데이터/애플리케이션
3,"쌍용정보통신, 해군전술C4I체계 성능 개량 사업 성료",http://www.itworld.co.kr/t/54649/%EB%B9%85%20%...,2020-09-29,IT관리/빅데이터/애플리케이션
4,"와이즈스톤, ‘인공지능품질연구포럼’ 발족…“인공지능 품질 표준화”",http://www.itworld.co.kr/t/54649/%EB%B9%85%20%...,2020-09-25,머신러닝/빅데이터/애플리케이션


In [17]:
# itworld prev data load
itw_prev = pd.read_csv('C:/driver/news_save/itw.csv',encoding='CP949') # ITWORLD 이전 기사 LOAD

In [18]:
itw_prev_title = itw_prev.title.tolist() # 이전 기사 제목 list 생성
itw_prev_list = [] # 중복 기사 판단 list
for row, now_news in enumerate(itw_df.title) :

    dupliacate_title(itw_prev_list,itw_prev_title) # 이전 날짜와 중복된 기사 제거

        
itw_df.drop(itw_prev_list,inplace=True)
itw_df

,title,url,date,tag
24,"넷앱, 소프트웨어 업데이트…“데이터센터에 클라우드 유연성 및 단순함 제공”",http://www.itworld.co.kr/t/34/%ED%81%B4%EB%9D%...,2020-10-22,클라우드
25,"티맥스A&C-효성인포메이션시스템, 클라우드 어플라이언스 시장 공략",http://www.itworld.co.kr/t/34/%ED%81%B4%EB%9D%...,2020-10-22,클라우드


In [19]:
itw_prev = pd.concat([itw_prev,itw_df]) # 오늘 날짜의 기사를 이전 날짜에 편입
itw_prev.to_csv('C:/driver/news_save/itw.csv',encoding='CP949',index=False) # 저장


In [20]:
itw_url =itw_df['url'].tolist()  ## 각 뉴스 URL 주소 입력
itw_article_list = []  # 뉴스 본문 저장 LIST 생성

for page in itw_url :
    
    res = requests.get(page, headers={'User-Agent':'Chrome/85.0.4183.121'},verify = False) ## 뉴스 URL 주소 접속
    news_html = BeautifulSoup(res.text,'html.parser') # html 태그 기준으로 parsing
    content = news_html.select(".node_body") # 기사 본문 저장되어 있는 html주소 복사


    news_content = content[0]


    final_article = str(news_content).replace('src="','src="http://www.itworld.co.kr').replace('&amp;apos;','').replace('\n','').replace('\t','')
    # 추출한 기사 전처리 
    final_article = final_article.replace('ⓒ', '').replace(u'\xa0',u'').replace('\u200b','').replace('\r','')
    # 추출한 기사 전처리 2
    article_list.append(final_article) # 전처리한 기사를 naver 뉴스 저장한 통합 list에 저장
    time.sleep(10)

final_title.extend(itw_df.title.tolist()) # 기사 제목을 naver 기사 저장한 통합 list에 저장

### ZDNet 인공지능 태그 크롤링

ZDNet의 경우 원격 접속이 막혀있어 현재 노트북의 크롬 user agent를 설정하여 접속하였습니다.  
향후 다른 사람의 컴퓨터로 사용하거나 크롬 버전이 변경될 경우 headers parameter에 'User-Agent'를 변경하여 사용해야 합니다.  



In [22]:
page = 'http://zdnet.co.kr/newskey/?lstcode=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5'
# zdnet 인공지능 url
res = requests.get(page, headers={'User-Agent':
                                  ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                                   "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36")},
                   verify = False)

news_html = BeautifulSoup(res.text,'html.parser') # html 태그 기준으로 parsing
content = news_html.select(".news_box") # 기사 본문 저장되어 있는 html주소 복사

zd_title_list =[] # 제목을 저장하기 위한 list
zd_url_list = [] # URL을 저장하기 위한 list
zd_date_list = [] # 날짜를 저장하기 위한 list
zdcontent = content[0].select('div > a') # 기사가 저장되어 있는 링크 추출

# 제목, 링크 출력 ---------------------------(2)
for result in zdcontent:
    if result.h3 == None: # 값이 존재하지 않을 경우 해당 루프 건너 뜀
        continue
    zd_title_list.append(str.strip(result.h3.string)) #기록된 기사 제목을 리스트에 추가
    zd_url_list.append('https://zdnet.co.kr' + result.attrs['href']) #기록된 기사 URL을 리스트에 추가
    
zd_date = content[0].select('.byline') # 날짜를 추출하기 위한 class select

# zdnet의 날짜는 yyyy.dd.mm 이후 AM, PM 이 기록되고, 이를 기준으로 다른 문자열이 나온다. 
# 이를 해결하기 위해 AM, PM기준으로 문자열을 자르고 앞의 문자열인 날짜를 남기고 전처리를 진행한다.

for date in zd_date : 
    date = date.span.string
    if 'AM' in date:
        date = str.strip(date.split("AM")[0]) # AM기준으로 split
    
    else : date = str.strip(date.split("PM")[0]) # PM기준으로 split
    date = date.replace('.','-')
    zd_date_list.append(date)

zd_df = pd.DataFrame({'title': zd_title_list,'url':zd_url_list,'date':zd_date_list}) # 기사제목과, URL을 dataframe에 임시 저장
zd_df['title'] = zd_df.title.str.replace('\\',"") # 따옴표가  \' 형태로 기록되어 해당 \를 공란으로 대체
zd_df = zd_df[~zd_df['title'].str.contains('MOU',na=False)] # MOU가 포함되어 있는 기사 제거
zd_df = zd_df.drop_duplicates(subset=['title'], keep='last').reset_index(drop=True) # 기사 중복 제거
zd_df.head()

,title,url,date
0,"화웨이, L4급 자율주행시스템 2년 내 상용화",https://zdnet.co.kr/view/?no=20201022080440,2020-10-22
1,"삼성 빅스비, 구글 클라우드 TPU로 학습속도 18배 빨라져",https://zdnet.co.kr/view/?no=20201021105528,2020-10-21
2,"KETI-SK텔레콤, 인공지능 라이다 시스템 상용화 맞손",https://zdnet.co.kr/view/?no=20201021104051,2020-10-21
3,"IT서비스 기업, AI기반 대외사업 본격화",https://zdnet.co.kr/view/?no=20201020173105,2020-10-21
4,"장동인 대표 ""AI도입, 외부 인력에만 의존하면 실패""",https://zdnet.co.kr/view/?no=20201020222444,2020-10-20


In [23]:
# zdnet prev data load
zd_prev = pd.read_csv('C:/driver/news_save/zd.csv',encoding='CP949')

In [24]:
zd_prev_title = zd_prev.title.tolist() # zdnet의 기사 제목 저장
zd_prev_list = []
for row, now_news in enumerate(zd_df.title) : # 이전 기사와 중복 제거 기준 생성
    if now_news in zd_prev_title :
        zd_prev_list.append(row) 
     
    else: continue
        
zd_df.drop(zd_prev_list,inplace=True) #기사 중복 제거

In [25]:
zd_prev = pd.concat([zd_prev,zd_df]) # 이전 기사에 오늘 날짜 기사 추가
zd_prev.to_csv('C:/driver/news_save/zd.csv',encoding='CP949',index=False) #저장

In [26]:
zd_df

,title,url,date
0,"화웨이, L4급 자율주행시스템 2년 내 상용화",https://zdnet.co.kr/view/?no=20201022080440,2020-10-22
1,"삼성 빅스비, 구글 클라우드 TPU로 학습속도 18배 빨라져",https://zdnet.co.kr/view/?no=20201021105528,2020-10-21
2,"KETI-SK텔레콤, 인공지능 라이다 시스템 상용화 맞손",https://zdnet.co.kr/view/?no=20201021104051,2020-10-21
3,"IT서비스 기업, AI기반 대외사업 본격화",https://zdnet.co.kr/view/?no=20201020173105,2020-10-21


#### ZDNet의 인공지능 태그는 매일 글이 올라오지 않는 경우가 많아 당일 기사가 없을 경우에는 SKIP합니다

In [27]:
zd_url =zd_df['url'].tolist()  ## 각 뉴스 URL 주소 입력


for page in zd_url :
    if not page:
        break

    res = requests.get(page, headers={'User-Agent':
                                      ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                                       "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36")},
                       verify = False) 
    ## 뉴스 URL 주소 접속
    news_html = BeautifulSoup(res.text,'html.parser') # html 태그 기준으로 parsing
    content = news_html.select("#content") # 기사 본문 저장되어 있는 html주소 복사


    zdcontent = content[0]


    for target in zdcontent.find_all('script'): # 기사 본문에 추가로 저장되어 있는 각 class들 제거
        target.extract()

    for target in zdcontent.find_all('section'): # 기사 본문에 추가로 저장되어 있는 각 class들 제거
        target.extract()


    for target in zdcontent.find_all('h2'): # 기사 본문에 추가로 저장되어 있는 각 class들 제거
        target.extract()

    for target in news_content.find_all('iframe'): # 기사 본문에 추가로 저장되어 있는 각 class들 제거
        target.extract()

    zdcontent = zdcontent.find_all(['p','figure']) # 기사 본문 추출

    zd_article ='' # 광고를 제거하고 기사 저장하기 위한 빈 문자열 생성
    for k in zdcontent :
        zd_article = zd_article + str(k) # 기사 에 해당 본문 추출한 tag 문자열 추가


    final_article = zd_article.replace('&amp;apos;',"").replace('\n','').replace('\t','').replace('\xa0','') #기사 본문 전처리

    article_list.append(final_article)
    time.sleep(10)
final_title.extend(zd_df.title.tolist())

In [28]:
print(len(final_title))
print(len(article_list))

8
8


#### 사이트 세 곳에서 기사를 가져왔고 네이버와 중복될 가능성이 있기 때문에
#### 기사제목을 종합하여 중복을 제거하고 진행합니다.

In [29]:
final_df = pd.DataFrame({'title':final_title, 'article':article_list}) # 기사제목과, URL을 dataframe에 임시 저장
final_df.drop_duplicates(subset=['title'], keep='first',inplace=True)
final_df.dropna(inplace=True)
final_title = final_df.title.tolist()
article_list= final_df.article.tolist()

### Selenium활용하여 Autoway에 자동 로그인 후 커뮤니티 게시
> IT기사 중 관련 없는 기사, 과학 기사 등 다양한 변수가 존재하여, 내부 검수가 필요하다고 판단하였습니다.  
>  
> 이를 위해 커뮤니티에 기사를 게시 후, 내부 검수하여 관련 없는 기사를 삭제하고, 
>  
> 일정 시간 이후에 남은 기사를 I.C.U에 게시하는 방향으로 정했습니다

In [30]:
path = "C:/driver/chromedriver.exe"    # 크롬드라이버 설치 경로 설정

set_id = open("C:/driver/set_id.txt", 'r').read() #id 입력
set_pwd = open("C:/driver/pwd.txt", 'r').read() # pwd 입력

In [90]:
# Selenium Load
driver = webdriver.Chrome(path) # chrome driver load
driver.maximize_window() # chrome driver 전체 화면 설정

# site login
driver.get('https://autoway.hyundai.net/') # 오토웨이 접속
time.sleep(3)

driver.find_element_by_xpath('//*[@id="form1"]/div[3]/div[2]/div[3]/div/div[3]').click() 
# autoway접속하여 로그인창의 부품 선택

driver.find_element_by_xpath('//*[@id="drpGroupCopList"]/option[2]').click() 
# 부품의 현대트랜시스 선택


log = driver.find_element_by_id("userID") # 오토웨이 id 입력 주소
log.send_keys(set_id) # id입력
time.sleep(2)
pwd = driver.find_element_by_id("password") # 오토웨이 비밀번호 입력 주소
pwd.send_keys(set_pwd) # pwd 입력
time.sleep(2)
driver.find_element_by_xpath('//*[@id="Login"]').click() # 로그인 버튼 클릭
time.sleep(5)

driver.get('https://autoway.hyundai.net/comm/C00621/Lists/B00002/AllItems.aspx') 
# 현대트랜시스 커뮤니티 test 주소 입력하여 접속


time.sleep(6)

for news_title, news_text in zip(final_title, article_list) :


    driver.find_element_by_xpath(('//*[@id="ctl00_ctl57_g_c18000eb_e182_4d10_b380_7481bde0914a_Search_'
                              'NewButton_New_ctl00_ctl57_g_c18000eb_e182_4d10_b380_7481bde0914a_Search_NewButton"]')).click()    
    # 커뮤니티 내부의 작성 버튼 클릭
    time.sleep(7)

    commu_title = driver.find_element_by_id("Title_fa564e0f-0c70-4ab9-b863-0177e6ddd247_$TextField") # 제목란 클릭

    commu_title.send_keys(news_title) # 작성 제목에 기사 제목 입력
    time.sleep(5)


    # iframe 전환

    # 작성 게시글 내부의 inline frame(iframe) 2개의 web page가 삽입되어 있습니다.
    # 기사를 입력하기 위해서는 내부에 들어가서 작성해야 합니다.
    # 이를 위해 tag_name으로 iframe이 입력된 전체 리스트를 찾고 
    # 기사 제목을 입력할 수 있는 iframe[0] 내부의 두번째 iframe에 접속하여 기사 본문을 게시하는 코드를 작성하였습니다.
    
    iframes = driver.find_elements_by_tag_name('iframe') # 모든 iframe tage_name을 검색
    driver.switch_to.frame(iframes[0]) # 그 중 본문을 작성할 수 있는 첫번째 iframe 선택
    time.sleep(2)

    second_iframe = driver.find_elements_by_tag_name('iframe') # 본문에 접속하기 위해서는 iframe[0]내부의 iframe에 접속하여 작성해야됨
    driver.find_element_by_xpath('//*[@id="ue_dext5editorsource"]/a').click() # I.C.U 본문작성 하위 태그의 HTML태그 클릭 
    # 이를 위해 2번째 iframe을 검색
    driver.switch_to.frame(second_iframe[0])  # 검색한 두 번째 iframe 접속
    commu_news = driver.find_element_by_id("dext_body") # 해당 iframe 내의 기사를 입력할 수 있는 id = dext_body 검색
    commu_news.clear()
    commu_news.send_keys(news_text) # 해당 부분에 기사 내용 입력
    time.sleep(5)
    
    driver.switch_to_default_content()# 해당 iframe에 벗어나 상위 웹페이지로 연결

    
    # 커뮤니티 게시
    driver.find_element_by_xpath('//*[@id="ctl00_ctl57_g_8e7ddc2e_52a0_4241_94ba_5da5a3edd86d_ctl00_ctl06_btnSave"]').click()
                                 
    time.sleep(3)


### 커뮤니티에서 내부한 검수 뉴스를 I.C.U에 게시하는 코드

In [91]:
# Selenium Load
driver = webdriver.Chrome(path) # chrome driver load
driver.maximize_window() # chrome driver 전체 화면 설정

# site login
driver.get('https://autoway.hyundai.net/') # 오토웨이 접속
time.sleep(3)

driver.find_element_by_xpath('//*[@id="form1"]/div[3]/div[2]/div[3]/div/div[3]').click() # autoway접속하여 로그인창의 부품 선택

driver.find_element_by_xpath('//*[@id="drpGroupCopList"]/option[2]').click() # 부품의 현대트랜시스 선택


log = driver.find_element_by_id("userID") # 오토웨이 id 입력 주소
log.send_keys(set_id) # id입력
time.sleep(2)
pwd = driver.find_element_by_id("password") # 오토웨이 비밀번호 입력 주소
pwd.send_keys(set_pwd) # pwd 입력
time.sleep(3)
driver.find_element_by_xpath('//*[@id="Login"]').click() # 로그인 버튼 클릭
time.sleep(7)

driver.get('https://autoway.hyundai.net/comm/C00621/Lists/B00002/AllItems.aspx')
time.sleep(6)

res = driver.page_source
community_html = BeautifulSoup(res,'html.parser') # html 태그 기준으로 parsing
content = community_html.select(".c-table__ellipsis") # 기사 본문 저장되어 있는 html주소 복사

inspect_list = []

# 추출한 태그 내부의 시스템 운영 ICT기획팀 등 다른 tag가 같이 저장됨 해당 tag가 나올 경우 skip

for i in content:
    if (i.string == '시스템운영') | (i.string == 'ICT기획팀') :
        continue
    com_str = i.string.replace('\xa0','')
    inspect_list.append(com_str)
    

com_row = []    
for row, title in enumerate(final_title) :
    if title not in inspect_list :
        com_row.append(row)

final_df.drop(com_row,inplace=True)
final_title = final_df.title.tolist()
article_list = final_df.article.tolist()


driver.get('https://autoway.hyundai.net/board/H104/Lists/H10400357/TileListView.aspx') # 현대트랜시스 I.C.U주소 입력하여 접속
time.sleep(6)

for final_icu_title, final_icu_article in zip(final_df.title, final_df.article) :

    driver.find_element_by_xpath('//*[@id="ctl00_ctl59_g_de894c14_b56d_4d4b_9c89_374a6c0455c1_Search_NewButton_New_ctl00_ctl59_g_de894c14_b56d_4d4b_9c89_374a6c0455c1_Search_NewButton"]/span').click()
    # I.C.U 내부의 작성 버튼 클릭
    time.sleep(7)

    icu_title = driver.find_element_by_id("Title_fa564e0f-0c70-4ab9-b863-0177e6ddd247_$TextField") # 제목란 클릭
    icu_title.send_keys(final_icu_title) # 작성 제목에 기사 제목 입력
    time.sleep(5)


    # iframe 전환

    # 작성 게시글 내부의 inline frame(iframe) 2개의 web page가 삽입되어 있습니다.
    # 기사를 입력하기 위해서는 내부에 들어가서 작성해야 합니다.
    # 이를 위해 tag_name으로 iframe이 입력된 전체 리스트를 찾고 
    # 기사 제목을 입력할 수 있는 iframe[0] 내부의 두번째 iframe에 접속하여 기사 본문을 게시하는 코드를 작성하였습니다.
    
    iframes = driver.find_elements_by_tag_name('iframe') # 모든 iframe tage_name을 검색
    driver.switch_to.frame(iframes[0]) # 그 중 본문을 작성할 수 있는 첫번째 iframe 선택
    time.sleep(2)

    second_iframe = driver.find_elements_by_tag_name('iframe') # 본문에 접속하기 위해서는 iframe[0]내부의 iframe에 접속하여 작성해야됨
    driver.find_element_by_xpath('//*[@id="ue_dext5editorsource"]/a').click() # I.C.U 본문작성 하위 태그의 HTML태그 클릭 
    # 이를 위해 2번째 iframe을 검색
    driver.switch_to.frame(second_iframe[0])  # 검색한 두 번째 iframe 접속
    icu_news = driver.find_element_by_id("dext_body") # 해당 iframe 내의 기사를 입력할 수 있는 id = dext_body 검색
    icu_news.clear()
    icu_news.send_keys(final_icu_article) # 해당 부분에 기사 내용 입력
    time.sleep(6)
    
    driver.switch_to_default_content()# 해당 iframe에 벗어나 상위 웹페이지로 연결
    time.sleep(1)
    # 기사 최종 게시 클릭

    driver.find_element_by_xpath('//*[@id="ctl00_ctl59_g_0005fe00_cee6_4765_9070_f77f554d027e_ctl00_ctl06_btnSave"]').click()
    

    
    # 임시저장 코드
#     driver.find_element_by_xpath('//*[@id="ctl00_ctl59_g_0005fe00_cee6_4765_9070_f77f554d027e_ctl00_ctl05_btnTempSave"]').click()
    time.sleep(3)